# Aprendizado Federado para reconhecimento de contexto em dispositivos móveis
## Experimento 1 - Pré-treinamento usual com uma base de usuários e Aprendizado Federado embarcado partindo do modelo pretreinado 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import es_utils as utils

2022-07-11 23:16:45.234036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 23:16:45.234254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 23:16:45.248304: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 23:16:45.248731: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-11 23:16:45.249122: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


Inicialmente precisamos separar quais 'labels' usaremos. Aqui, optamos pelas 51 labels que segundo artigo do ExtraSensosy utiliza, de modo a ter bons parâmetros de comparação

In [3]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

"labels = ['label:LYING_DOWN',\n 'label:SITTING',\n 'label:FIX_walking',\n 'label:FIX_running',\n 'label:BICYCLING',\n 'label:SLEEPING',\n 'label:LAB_WORK',\n 'label:IN_CLASS',\n 'label:IN_A_MEETING',\n 'label:LOC_main_workplace',\n 'label:OR_indoors',\n 'label:OR_outside',\n 'label:IN_A_CAR',\n 'label:ON_A_BUS',\n 'label:DRIVE_-_I_M_THE_DRIVER',\n 'label:DRIVE_-_I_M_A_PASSENGER',\n 'label:LOC_home',\n 'label:FIX_restaurant',\n 'label:PHONE_IN_POCKET',\n 'label:OR_exercise',\n 'label:COOKING',\n 'label:SHOPPING',\n 'label:STROLLING',\n 'label:DRINKING__ALCOHOL_',\n 'label:BATHING_-_SHOWER',\n 'label:CLEANING',\n 'label:DOING_LAUNDRY',\n 'label:WASHING_DISHES',\n 'label:WATCHING_TV',\n 'label:SURFING_THE_INTERNET',\n 'label:AT_A_PARTY',\n 'label:AT_A_BAR',\n 'label:LOC_beach',\n 'label:SINGING',\n 'label:TALKING',\n 'label:COMPUTER_WORK',\n 'label:EATING',\n 'label:TOILET',\n 'label:GROOMING',\n 'label:DRESSING',\n 'label:AT_THE_GYM',\n 'label:STAIRS_-_GOING_UP',\n 'label:STAIRS_-_GOING

In [4]:
labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]

len(labels)

6

Separamos os 60 usuários em 5 pastas onde cada pasta considera 40 usuários selecionados aleatoriamente para treino e os 20 restantes para teste. Os dados dos 20 usuários são ainda separados em 4 arquivos *.csv* (x_train, x_test, y_train, y_test) e colocados em pastas nomeadas com o uuid do usuário.

In [5]:
k_folds = utils.create_k_folds_n_users(5, 40, '/home/nonroot/sample_data')

/home/nonroot/es_utils.py:518: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:518: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:518: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:518: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:518: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

Em seguida, para cada pasta, treinamos um modelo de MLP fazendo uma otimização de hiperparâmetros e convertemos em um modelo *tflite*. Os hiperparâmetros otimizados são:

- número de camadas escondidas (*hidden layers*): 1 ou 2
- número de neurônios em cada camada escondida: 4 a 64, com um passo de tamanho 4
- taxa de aprendizado (*learning rate*) do otimizador (*Adaptive Moment Estimation*, **Adam**): 1e-1, 1e-2, 1e-3 ou 1e-4

In [6]:

for fold in k_folds:
    config = {
        'df_path': k_folds[fold]['40'],
        'labels': labels
    }
    
    har = utils.HAR(config)
    #har.run()
    har.hypertunning()
    
    har.mlp.model.save(f'./model/saved_model_{fold}')
    converter = tf.lite.TFLiteConverter.from_saved_model(f'./model/saved_model_{fold}') # path to the SavedModel directory
    tflite_model = converter.convert()
    with open(f'./model/model_base_{fold}.tflite', 'wb') as f:
      f.write(tflite_model)

X train-test shape: (242377, 226) - X test shape: (60595, 226)
y train-test shape: (242377, 6) - y test shape: (60595, 6)
INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json
INFO:tensorflow:Reloading Tuner from hypertunning/experimento_1_RandSearchCV/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
  44/6060 [..............................] - ETA: 14s - loss: 0.5037 - avg_multilabel_BA_2: 0.6407  

2022-07-11 23:21:43.630185: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


6060/6060 [==============================] - 20s 3ms/step - loss: 0.2399 - avg_multilabel_BA_2: 0.7140 - val_loss: 0.2414 - val_avg_multilabel_BA_2: 0.6819
Epoch 2/100
6060/6060 [==============================] - 17s 3ms/step - loss: 0.2445 - avg_multilabel_BA_2: 0.6717 - val_loss: 0.2386 - val_avg_multilabel_BA_2: 0.6661
Epoch 3/100
6060/6060 [==============================] - 17s 3ms/step - loss: 0.2891 - avg_multilabel_BA_2: 0.6534 - val_loss: 0.3132 - val_avg_multilabel_BA_2: 0.6290
Epoch 4/100
6060/6060 [==============================] - 17s 3ms/step - loss: 0.3141 - avg_multilabel_BA_2: 0.6105 - val_loss: 0.3131 - val_avg_multilabel_BA_2: 0.5959
Epoch 5/100
6060/6060 [==============================] - 17s 3ms/step - loss: 0.3140 - avg_multilabel_BA_2: 0.5852 - val_loss: 0.3135 - val_avg_multilabel_BA_2: 0.5764
Epoch 6/100
6060/6060 [==============================] - 16s 3ms/step - loss: 0.3140 - avg_multilabel_BA_2: 0.5694 - val_loss: 0.3137 - val_avg_multilabel_BA_2: 0.5634
Epoc

Com os modelos salvos, o restante do experimento segue em ambiente Android